In [ ]:
# IPython magic to import matplotlib and plot inline
%matplotlib inline

In [ ]:
# Path settings
import os
# we use a relative path here, you can also specify an absolute path for your system
out_path = "output/04_hyperelasticity_ogden"
os.makedirs(out_path, exist_ok=True)

# Hyperelasticity Ogden Model

Solves a simple mechanical problem with Ogden hyperelastic material model.
The strain-energy density function is expressed in terms of the stretch ratios $\lambda_i$ which are computed as the eigenvalues of the Right Cauchy-Green tensor using the approach outlined in notebook [03_computing_eigenvalues_ufl.ipynb](https://github.com/danielabler/femII_fenics_2019/03_computing_eigenvalues_ufl.ipynb) in this repository. 

In [ ]:
import fenics as fe
import os

In [ ]:
# mesh
mesh = fe.BoxMesh(fe.Point(0,0,0),fe.Point(10.0,1,1),100,10,10)
mesh

In [ ]:
# FunctionSpace
V = fe.VectorFunctionSpace(mesh, "Lagrange", 1)

In [ ]:
# Finite element functions
du = fe.TrialFunction(V)
v  = fe.TestFunction(V)
u  = fe.Function(V)

In [ ]:
# Mark boundary subdomians
left =  fe.CompiledSubDomain("near(x[0], side) && on_boundary", side = 0.0)

# Define Dirichlet BC: zero displacement on left side (x = 0)
c = fe.Expression(("0.0", "0.0", "0.0"), element=V.ufl_element())
bcl = fe.DirichletBC(V, c, left)
bcs = [bcl]


# Define Von Neuman BC: surface load on right side (x=10)
#-- a) function to define boundary
class RightBorder(fe.SubDomain):
    def inside(self, x, on_boundary):
        return x[0] >= 10.0-fe.DOLFIN_EPS and on_boundary
#-- b) mark up boundary in meshfunction
boundaries = fe.MeshFunction("size_t", mesh, mesh.topology().dim() - 1)
boundaries.set_all(0)
right_border = RightBorder()
right_border.mark(boundaries, 1)

#-- c) compute surface load over buondary
dsp    = fe.Measure('ds', domain=mesh, subdomain_data=boundaries)
load_s = fe.Constant((100., 0., 0.))
# term in variational form
integral_S = fe.inner(load_s, u) * dsp(1)


#-- save boundary to vtu for inspection
boundary_file = fe.File(os.path.join(out_path, "boundaries.pvd"))
boundary_file << boundaries

In [ ]:
#### Functions for 

def defGrad(u):
    """
    Compute deformation gradient from displacements
    """
    d = u.geometric_dimension()
    I = fe.Identity(d)
    F = I + fe.grad(u)             
    return F

import numpy as np

# Invariants 
def I1(T):
    return fe.tr(T)

def I2(T):
    t1 = T[0,0]*T[1,1] + T[0,0]*T[2,2] + T[1,1]*T[2,2]
    t2 = T[0,1]*T[0,1] + T[0,2]*T[0,2] + T[1,2]*T[1,2]
    return t1 - t2

def I3(T):
    return fe.det(T)

def v_inv(T):
    return (I1(T)/3.)**2 - I2(T)/3.

def s_inv(T):
    return (I1(T)/3.)**3 - I1(T)*I2(T)/6. + I3(T)/2.

def phi_inv(T):
    arg = s_inv(T)/v_inv(T)*fe.sqrt(1./v_inv(T))
    # numerical issues if arg~0 -> https://fenicsproject.org/qa/12299/nan-values-when-computing-arccos-1-0-bug/
    arg_cond = fe.conditional( fe.ge(arg,  1-fe.DOLFIN_EPS),  
                                     1-fe.DOLFIN_EPS, 
                                     fe.conditional( fe.le(arg, -1+fe.DOLFIN_EPS), -1+fe.DOLFIN_EPS, arg ))
    return fe.acos(arg_cond)/3.

def strainEnergyDensityFunctionOgden(F, mu, alpha):
    C = F.T*F      # Right Cauchy-Green tensor
    lambda_1 = I1(C)/3. + 2*fe.sqrt(v_inv(C))*fe.cos(phi_inv(C))
    lambda_2 = I1(C)/3. - 2*fe.sqrt(v_inv(C))*fe.cos(fe.pi/3. + phi_inv(C))
    lambda_3 = I1(C)/3. - 2*fe.sqrt(v_inv(C))*fe.cos(fe.pi/3. - phi_inv(C))
    # eigenvalue is stretchratio^2 -> lambda^(alpha/2)
    return 2*mu*(lambda_1**(alpha/2.) + lambda_2**(alpha/2.) + lambda_3**(alpha/2.) - 3) / alpha**2
    
def strainEnergyDensityFunctionNeoHookean(F, mu, lmbda):
    """
    Computes strain energy density of neo hookean material model in function o f
    - F: deformation gradient
    - Ey: young's modulus
    - nu: poisson ratio
    """
    C = F.T*F      # Right Cauchy-Green tensor
    I1 = fe.tr(C)  # Invariants 
    J = fe.det(F)
    #return (mu/2)*(I1 - 3) - mu*fe.ln(J) + (lmbda/2)*(fe.ln(J))**2
    return (mu/2)*(I1 - 3) # this formulation *should* be identical to the Ogden model with alpha=2


In [ ]:

# create deformation gradient
F = defGrad(u)

F = fe.variable(F) # !!! needed to be able to differentiate strain energydensity function wrt F for stress computation

# instantiate strain energy density function for given material properties
psi = strainEnergyDensityFunctionOgden(F=F, mu=360, alpha=2)

#psi = strainEnergyDensityFunctionNeoHookean(F=F, mu=360, lmbda=1400)


# body force
B = fe.Constant((0., 0., 0.))

Pi = psi * fe.dx - fe.inner(B, u) * fe.dx - integral_S

# Compute 1st variation of Pi (directional derivative about u in dir. of v)
Fpi = fe.derivative(Pi, u, v)

# Compute Jacobian of F
Jac = fe.derivative(Fpi, u, du)

As we have seen in [03_computing_eigenvalues_ufl.ipynb](https://github.com/danielabler/femII_fenics_2019/03_computing_eigenvalues_ufl.ipynb), our approach for computing the $\lambda_i$ does not work if the matrix is already diagonalized.
By default, the displacement field `u` is initialized as (0,0,0) everywhere, which implies that `F` is the identity matrix and so is `C`.
As a result (and erroneously), the eigenvalues cannot be computed and the solution of the variational problem fails.
Initializing `u` with random values solves this issue.

In [ ]:
# need non-zero starting values for displacement field
# if zero displacement field -> C=id -> v_inv=0 -> division by 0 in phi_inv
n = u.vector().get_local().shape
import numpy as np
u_noise_np = np.random.rand(n[0])
u.vector().set_local(u_noise_np)

In [ ]:
# Define the solver
problem = fe.NonlinearVariationalProblem(Fpi, u, bcs, Jac)
solver = fe.NonlinearVariationalSolver(problem)

# Set solver parameters (optional)
prm = solver.parameters
prm['nonlinear_solver'] = 'newton'
prm['newton_solver']['linear_solver'] = 'cg'
prm['newton_solver']['preconditioner'] = 'icc'

prm['newton_solver']['error_on_nonconvergence'] = True
prm['newton_solver']['absolute_tolerance'] = 1E-9
prm['newton_solver']['relative_tolerance'] = 1E-8
prm['newton_solver']['maximum_iterations'] = 50
prm['newton_solver']['relaxation_parameter'] = 1.0

prm['newton_solver']['lu_solver']['report'] = True
#prm['newton_solver']['lu_solver']['reuse_factorization'] = False
#prm['newton_solver']['lu_solver']['same_nonzero_pattern'] = False
prm['newton_solver']['lu_solver']['symmetric'] = False

prm['newton_solver']['krylov_solver']['error_on_nonconvergence'] = True
prm['newton_solver']['krylov_solver']['absolute_tolerance'] = 1E-7
prm['newton_solver']['krylov_solver']['relative_tolerance'] = 1E-5
prm['newton_solver']['krylov_solver']['maximum_iterations'] = 1000
prm['newton_solver']['krylov_solver']['nonzero_initial_guess'] = True

solver.solve()

In [ ]:
# Save results
Z = fe.TensorFunctionSpace(mesh, 'P', 1)
W = fe.FunctionSpace(mesh, "P", 1)
X = fe.VectorFunctionSpace(mesh, "P", 1)

u.rename("displacement", "")

print('Computing stress ...')
# Computation of the stresses
S = fe.diff(psi, F)            # compute stress by differentiation of psi!
S_project = fe.project(S, Z, solver_type='cg', preconditioner_type='icc')
S_project.rename("stress", "")

print('Computing van Mises Stress ...')
# Computation of van Mises Stress
s = S - (1./3) * fe.tr(S) * fe.Identity(u.geometric_dimension())
von_Mises = fe.sqrt( 3./2 * fe.inner(s, s) )
von_Mises_project = fe.project(von_Mises, W, solver_type='cg', preconditioner_type='icc')
von_Mises_project.rename("van_mises", "")

print('Computing Cauchy-Green tensor ...')
# Right Cauchy-Green tensor
F_sol = defGrad(u)
C_sol = F_sol.T*F_sol     
cauchy_green = fe.project(C_sol, Z, solver_type='cg', preconditioner_type='icc')
cauchy_green.rename("cauchy_green", "")

# potentials
psi_ogden = strainEnergyDensityFunctionOgden(F=F_sol, mu=360, alpha=2)
psi_ogden_fun = fe.project(psi_ogden, W, solver_type='cg', preconditioner_type='icc')
psi_ogden_fun.rename("psi_Ogden", "")

psi_neohook = strainEnergyDensityFunctionNeoHookean(F=F_sol, mu=360, lmbda=1400)
psi_neohook_fun = fe.project(psi_neohook, W, solver_type='cg', preconditioner_type='icc')
psi_neohook_fun.rename("psi_NeoHook", "")


In [ ]:
# Compute eigenvalues
def eigen_value(T, i):
    if i==1:
        ev =  I1(T)/3. + 2*fe.sqrt(v_inv(T))*fe.cos(phi_inv(T))
    elif i==2:
        ev =  I1(T)/3. - 2*fe.sqrt(v_inv(T))*fe.cos(fe.pi/3. + phi_inv(T))
    elif i==3:
        ev =  I1(T)/3. - 2*fe.sqrt(v_inv(T))*fe.cos(fe.pi/3. - phi_inv(T))
    return ev

def eigen_vector(T, i):
    eval_i = eigen_value(T, i)
    A = T[0,0] - eval_i
    B = T[1,1] - eval_i
    C = T[2,2] - eval_i
    
    evec_ix = (T[0,1]*T[1,2] - B*T[0,2]) * (T[0,2]*T[1,2] - C*T[0,1]) # - C*T[0,1] -> not fully typed in paper, there might be an error!
    evec_iy = (T[0,2]*T[1,2] - C*T[0,1]) * (T[0,2]*T[0,1] - A*T[1,2]) 
    evec_iz = (T[0,1]*T[1,2] - B*T[0,2]) * (T[0,2]*T[0,1] - A*T[1,2]) 
    
    evec = fe.as_vector( (evec_ix, evec_iy, evec_iz) )
    
    evec_norm = evec / fe.sqrt(fe.inner(evec,evec))
    return evec_norm

evals_ufl = []
evecs_ufl = []

for i in [1,2,3]:
    print("Computing EV %i"%i)
    eval_i = eigen_value(C_sol, i)
    eval_field = fe.project(eval_i, W, solver_type='cg', preconditioner_type='icc')
    eval_field.rename("lambda_%i"%i, "")
    evals_ufl.append(eval_field)
    
    evec_i = eigen_vector(C_sol, i)
    evec_field = fe.project(evec_i, X, solver_type='cg', preconditioner_type='icc')
    evec_field.rename("ev_%i"%i, "")
    evecs_ufl.append(evec_field)


In [ ]:
# a more compact way of writing all results into the same file:

xdmffile = fe.XDMFFile(os.path.join(out_path, 'results.xdmf'))
xdmffile.parameters["flush_output"] = True
xdmffile.parameters["functions_share_mesh"] = True

xdmffile.write(S_project,0)
xdmffile.write(von_Mises_project,0)
xdmffile.write(u,0)
xdmffile.write(cauchy_green,0)
xdmffile.write(psi_ogden_fun,0)
xdmffile.write(psi_neohook_fun,0)


for i in range(0,3):
    xdmffile.write(evecs_ufl[i],0)
    xdmffile.write(evals_ufl[i],0)

xdmffile.close()

NOTE:
- For identical parameter settings (mu, lambda=2) both formulations (using stretch ratios [ogden] vs trace [neohookean]) seem to give the same results
- test this by changing the strain density function used for computation; make sure to restart the kernel between tests.